In [1]:
!pip install rdkit

In [1]:
CUDA_LAUNCH_BLOCKING = "1"

import torch
import torchvision
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
#from google.colab import drive
import math
import rdkit
from rdkit.Chem import AllChem as Chem
from rdkit.Chem import QED
from rdkit.Chem import MolSurf
from rdkit.Chem import rdFreeSASA as SASA
import os
import seaborn as sb
from dataframe_pipeline_script import Dataset
#drive.mount('/content/drive')


In [2]:
#path = "/content/drive/My Drive/Toxicity Project/DATA/fingerprint_df.csv"
path = "DATA/fingerprint_df.csv"
df = pd.read_csv(path)
df.head()

,index,Unnamed: 0.1,Unnamed: 0,id,title,common_name,description,cas,pubchem_id,chemical_formula,...,fp122,fp123,fp124,fp125,fp126,fp127,fp128,binary_rat_ld50,LD50_level,LD50_rat_level
0,0,0,0,2,T3D0001,Arsenic,Arsenic(As) is a ubiquitous metalloid found in...,7440-38-2,104734.0,As,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0
1,1,1,3,5,T3D0004,Vinyl chloride,"Vinyl chloride is a man-made organic compound,...",75-01-4,6338.0,C2H3Cl,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,2.0
2,2,2,4,7,T3D0006,Benzene,"Benzene is a toxic, volatile, flammable liquid...",71-43-2,241.0,C6H6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,3.0
3,3,3,5,8,T3D0007,Cadmium,Cadmium (group IIB of the periodic table of el...,7440-43-9,31193.0,Cd,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0
4,4,5,9,13,T3D0012,Clofenotane,Insecticide. Clofenotane is a major component ...,50-29-3,3036.0,C14H9Cl5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [3]:
B = 8 #Reduce this if your GPU runs out of memory
valset = Dataset(data=df, label="LD50_rat", purpose="val", shuffle=True,representation="both", qed=False, max_len=125)
valloader = torch.utils.data.DataLoader(dataset=valset,batch_size=B,shuffle=False,drop_last=True)

trainset = Dataset(data=df, label="LD50_rat", purpose="train", shuffle=True,representation="both", qed=False, max_len=125)
trainloader = torch.utils.data.DataLoader(dataset=trainset,batch_size=B,shuffle=False,drop_last=True)



Val
Length of longest string: 125
ignored: 1
token lengths: 126
finger lengths: 128
Largest value: 124.0
Number of unique embedding values: 51
x shape: torch.Size([169, 254])
t shape: torch.Size([169])

Train
Length of longest string: 125
ignored: 0
token lengths: 126
finger lengths: 128
Largest value: 124.0
Number of unique embedding values: 47
x shape: torch.Size([394, 254])
t shape: torch.Size([394])


[19:21:45] Explicit valence for atom # 0 O, 3, is greater than permitted


In [4]:
# you can access initialised variables from the Dataset class by calling the variables as arguments of the instance:
print(trainset.n_map)
print(trainset.max_map)
print(valset.max_len)

47
124.0
125


In [5]:
### GPU type
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon May 22 19:21:47 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 522.06       Driver Version: 522.06       CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:10:00.0  On |                  N/A |
| 44%   55C    P8    31W / 350W |   2937MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
class Head(torch.nn.Module):
    def __init__(self,head_size, n_embd):
        super().__init__()
        self.key = torch.nn.Linear(int(n_embd), int(head_size), bias=False)
        self.query = torch.nn.Linear(int(n_embd), int(head_size), bias=False)
        self.value = torch.nn.Linear(int(n_embd), int(head_size), bias=False)
        self.dropout = torch.nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x)
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = torch.nn.functional.softmax(wei,dim=-1) #softmax needs to be performed along the sequence axis (_,seq,_) 
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class CrossHead(torch.nn.Module): 
    def __init__(self,head_size, n_embd):
        super().__init__()
        self.key = torch.nn.Linear(int(n_embd), int(head_size), bias=False)
        self.query = torch.nn.Linear(int(n_embd), int(head_size), bias=False)
        self.value = torch.nn.Linear(int(n_embd), int(head_size), bias=False)
        self.dropout = torch.nn.Dropout(dropout)

    def forward(self, x, y_enc): # here the keys and the values come from the encoder block
        B,T,C = x.shape
        k = self.key(y_enc)   # (B,T,C)
        q = self.query(x)
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = torch.nn.functional.softmax(wei,dim=-1) #softmax needs to be performed along the sequence axis (_,seq,_) 
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(y_enc) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out    

class MultiHeadAttention(torch.nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size, n_embd):
        super().__init__()
        self.heads = torch.nn.ModuleList([Head(int(head_size), int(n_embd)) for _ in range(num_heads)])
        self.proj = torch.nn.Linear(int(n_embd), int(n_embd))
        self.dropout = torch.nn.Dropout(dropout)
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) #
        out = self.dropout(self.proj(out))
        return out  
    
class MultiHeadCrossAttention(torch.nn.Module):
    """ multiple heads of cross-attention in parallel """

    def __init__(self, num_heads, head_size, n_embd):
        super().__init__()
        self.heads = torch.nn.ModuleList([CrossHead(int(head_size), int(n_embd)) for _ in range(num_heads)])
        self.proj = torch.nn.Linear(int(n_embd), int(n_embd))
        self.dropout = torch.nn.Dropout(dropout)
    def forward(self, x, y_enc):
        out = torch.cat([h(x, y_enc) for h in self.heads], dim=-1) #
        out = self.dropout(self.proj(out))
        return out

class FeedForward(torch.nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = torch.nn.Sequential(
            torch.nn.Linear(int(n_embd), 4 * int(n_embd)),
            torch.nn.ReLU(), #investigate GeLU instead of ReLU
            torch.nn.Linear(4 * int(n_embd), int(n_embd)),
            torch.nn.Dropout(dropout), #dropout helps with overfitting
        )

    def forward(self, x):
        return self.net(x)
    
class Encoder(torch.nn.Module): # encoder needed for when  have the full data (maybe)
    """Encoder block: communication followed by computation """

    def __init__(self, n_embd, n_head, head_size):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        self.sa = MultiHeadAttention(n_head, head_size, n_embd)
        self.ffwd = FeedForward(int(n_embd))
        self.ln1 = torch.nn.LayerNorm(int(n_embd))
        self.ln2 = torch.nn.LayerNorm(int(n_embd)) #layernorm is not switched off at eval. so it is not buffered 

        
    def forward(self, x):
        x = x + self.sa(self.ln1(x)) #in residual connection, addition distributes gradients equally to all of its branches.    
        #consider switching the Layernorm to after the computation
        x = x + self.ffwd(self.ln2(x)) #LayerNorm before the transformation instead of after avoids the instability of the training due to large output gradients 
        return x

class Decoder(torch.nn.Module): # encoder needed for when  have the full data (maybe)
    """Decoder block: communication followed by computation """

    def __init__(self, n_embd, n_head, head_size):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        
        self.ca= MultiHeadCrossAttention(n_head, head_size, n_embd)
        self.ffwd = FeedForward(int(n_embd))
        self.ln1 = torch.nn.LayerNorm(int(n_embd))
        self.ln_enc = torch.nn.LayerNorm(int(n_embd))
        self.ln2 = torch.nn.LayerNorm(int(n_embd))
        
    def forward(self, x, y_enc):
#in residual connection, addition distributes gradients equally to all of its branches.    
        x = x + self.ca(self.ln1(x), self.ln_enc(y_enc)) #consider switching the Layernorm to after the computation  
        x = x + self.ffwd(self.ln2(x)) #LayerNorm before the transformation instead of after avoids the instability of the training due to large output gradients 
        return x
    
class PositionalEncoding(torch.nn.Module):
    def __init__(self, n_embd, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = torch.nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, n_embd)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, n_embd, 2).float() * (-math.log(10000.0) / n_embd))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(1),:] #consider removing dropout and plotting the layer
        return x#self.dropout(x)

In [7]:
class ClassificationTransformer(torch.nn.Module):

    def __init__(self,n_embd,n_head,n_layers,n_class):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        head_size = int(n_embd) // n_head
        self.token_embedder = torch.nn.Embedding(int(trainset.max_map+1), n_embd)
        self.position_embedder = PositionalEncoding(n_embd)
        self.property_embedder = torch.nn.Linear(128,n_embd)
        self.EncoderBlock = torch.nn.Sequential(*[Encoder(n_embd=n_embd, n_head=n_head, head_size=head_size) for _ in range(n_layers)])
        self.Decoder1 = Decoder(n_embd=n_embd, n_head=n_head, head_size=head_size)
        #self.Decoder2 = Decoder(n_embd=n_embd, n_head=n_head, head_size=head_size)
        self.fc_map = torch.nn.Linear(n_embd,n_class)
        self.softmax = torch.nn.Softmax()
        
        
    def forward(self, x):
        props = x[:,-128:]
        props = torch.unsqueeze(props,1)
        x = x[:,:-128]
        B, T = x.shape
        # idx and targets are both (B,T) tensor of integers
        token_emb = self.token_embedder(x.long())
        x = self.position_embedder(token_emb) # (B,T,C)
        props = self.property_embedder(props)
        x_enc = self.EncoderBlock(x) # (B,T,C)
        x = self.Decoder1(props,x_enc)
        #x = self.Decoder2(x,x_enc)
        y = self.fc_map(x)
        #y = self.softmax(x)
        y = torch.squeeze(y)
        return y

In [8]:
dropout = 0.8
n_embd = 512
n_head = 2
n_layers = 1
transformer = ClassificationTransformer(n_embd=n_embd,n_head=n_head,n_layers=n_layers,n_class=1)
total_params = sum(param.numel() for param in transformer.parameters())

print(f"Total model parameters: {total_params}")

Total model parameters: 6433281


In [9]:
device = torch.device("cuda")
optimizer = torch.optim.AdamW(params=transformer.parameters(), lr=0.0001)
loss = torch.nn.L1Loss()
transformer = transformer.to(device)
train_plot = []
val_plot = []
best = 100000
train_loader = trainloader
val_loader = valloader
train_set = trainset
val_set = valset
for epoch in tqdm(range(50)):
    train_loss = val_loss = 0
    transformer.train()
    for x,t in tqdm(train_loader):
        
        
        x = x.float()
        t = t.float()
        optimizer.zero_grad()
        data = transformer(x.to(device))
        J = loss(data, t.to(device))
        train_plot.append(J.item())
        train_loss += J.item()
        J.backward()
        optimizer.step()
        
    transformer.eval()
    for x,t in tqdm(val_loader):
        with torch.no_grad():
            
            x = x.float()
            t = t.float()
            data = transformer(x.to(device))
            J = loss(data, t.to(device))
            val_loss += J.item()
            val_plot.append(J.item())
    if (val_loss/len(val_loader)) < best:
        best = (val_loss/len(val_loader))
        torch.save(transformer.state_dict(), f"Saved_models/toxformer_regressor_encoderdecoder_params.pth") ### saving the model with best validation loss is a bias!!!
    print(f"\rEpoch {epoch+1}; train loss: {train_loss/len(train_loader):1.5f}\t val loss: {val_loss/len(val_loader):1.5f}")
##### print the loss calcualted by hand (custom function) more reliable
R=B*5
ma_train = np.convolve(train_plot, np.ones(R)/R, mode="valid")
ma_val = np.convolve(val_plot, np.ones(R)/R, mode="valid")
plt.plot(np.linspace(0,len(train_plot)-1,len(ma_train)),ma_train,c="tab:orange")
plt.plot(np.linspace(0,len(train_plot)-1,len(ma_val)),ma_val, c="tab:blue")
plt.plot(train_plot,c="tab:orange",alpha=0.3)
plt.plot(np.linspace(0,len(train_plot)-1,len(val_plot)),val_plot,c="tab:blue",alpha=0.3)


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/49 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

Epoch 1; train loss: 1039.49497	 val loss: 2071.47763


  0%|          | 0/49 [00:00<?, ?it/s]

...

  0%|          | 0/49 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

Epoch 16; train loss: 483.70657	 val loss: 2123.26316


  0%|          | 0/49 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [35]:
transformer.load_state_dict(torch.load("Saved_models/toxformer_regressor_encoderdecoder_params.pth"))

<All keys matched successfully>

In [37]:
np.mean(np.abs(df.LD50_rat-np.mean(df.LD50_rat)))

1251.0875945374999

In [38]:
### validation and evaluation code. something is wrong with the MSE loss. if computed manualy the mse loss differs from the calcualtion performed by pytorch.
device = torch.device("cuda")
transformer.to(device)
pred = []
test = []
pred0 = []
test0 = []
pars0 = []

correct = 0
total = 0

mse = torch.nn.MSELoss()
mae = torch.nn.L1Loss()
maeloss = 0
mseloss = 0
loader = valloader     #fullval_loader
transformer.eval()
with torch.no_grad():
    for x,t in tqdm(loader):
        x = x.float()
        t = t.float()
        predicted = transformer(x.to(device))
        total += t.size(0)
        correct += (predicted == t.to(device)).sum().item()


        data = torch.flatten(data)
        t = torch.flatten(t)
        # pars0.append((torch.permute(x[:,-7:],(0,2,1))).cpu().detach().numpy()) 
        inz = torch.nonzero(t).squeeze()
        J_mse = mse(data,t.to(device))
        J_mae = mae(data, t.to(device))

        mseloss =+ J_mse.item()
        maeloss =+ J_mae.item()
        test.append(t.cpu().detach().numpy())
        pred.append(data.cpu().detach().numpy())

print(f'Accuracy of the network: {100 * correct / total} %')

print(f"Validation MSE loss: {mseloss/len(val_loader):0.5f}")
print(f"\nValidation MAE loss: {maeloss/len(val_loader):0.5f}")

pred = np.concatenate(pred)
test = np.concatenate(test)


  0%|          | 0/21 [00:00<?, ?it/s]

Accuracy of the network: 0.0 %
Validation MSE loss: 3959.37872

Validation MAE loss: 12.51884


In [184]:
test_df = pd.read_csv("DATA/test_df.csv")

In [185]:
(test_df.toxicity.isna().astype(int)==0).astype(int).sum()

16